# Part 1

In [ ]:
import math

with open("input.txt") as input:
  lines = input.readlines()

coords = [list(map(int,x.split(","))) for x in lines] # 97989,50290 

part1 = 0

for a in coords:
  for b in coords: 
    base = abs(a[0]-b[0])+1
    height = abs(a[1]-b[1])+1
    area = base * height
    if area > part1:
      part1 = area

print("Part 1 Answer",part1) 


# Part 2

## Quick author note: 

Solving this problem turned out to be extremely hard, mostly because of my pride and my refusal to seek help online. When I finally swallowed that pride and started reading other people’s solutions and pseudocode, it was just as hard and it took another couple of hours before I realized my mistake. 

Normally, I like to clean up my code before pushing it. This time, I didn’t. I wanted to share the raw insanity that went into solving this problem.

Enjoy. Or don’t.
Here it is.

Total hours spent: 10+

In [ ]:
from functools import lru_cache

with open("input.txt") as input:
  lines = input.readlines()

red_tiles = [tuple(map(int,x.split(","))) for x in lines]
# Add final to complete the loop
red_tiles.append(red_tiles[0])
red_tiles_set = set(red_tiles)

green_tiles = []
horizontal_lines = [] # (x1,x2,y)
vertical_lines = [] # (y1,y2,x) 
all_lines = []
tile_on_perimeter = set()

# Storing all lines
# Change coord system (x1,y1,x2,y2)
for i in range(len(red_tiles)-1):
  t1, t2 = red_tiles[i], red_tiles[i+1]
  all_lines.append((t1[0],t1[1],t2[0],t2[1]))

# Function to traverse the perimeter of a triangle
# Check whole line instead of every singular element!!!!!! 
def get_rectangular_perimeter(corner_a, corner_b): # corner = (x,y) => [(x,y)]
  a_x, a_y = corner_a
  b_x, b_y = corner_b
  all_x, all_y = set(),set()
  for x in range(min(a_x,b_x)+1,max(a_x,b_x)):
    all_x.add((x,a_y))
    all_x.add((x,b_y))
  for y in range(min(a_y,b_y)+1,max(a_y,b_y)):
    all_y.add((a_x,y))
    all_y.add((b_x,y))
  return list(all_x.union(all_y))

# Get all tiles on the perimeter of the polygon
for line in all_lines:
  a,b = (line[0],line[1]),(line[2],line[3])
  tile_on_perimeter.update(get_rectangular_perimeter(a,b))

@lru_cache(maxsize=None)
def tile_inside_polygon(tile): # tile = (x,y)
  
  # # Check if tile inside line and line over 
  # lines_above = [x for x in all_lines if tile[0] > min(x[0],x[2]) and tile[0] < max(x[0],x[2]) and tile[1] > max(x[1],x[3])]
  # if len(lines_above) > 0 and len(lines_above) % 2 == 0:
  #   return False

  # # Check if tile inside lines and under over
  # lines_under = [x for x in all_lines if tile[0] > min(x[0],x[2]) and tile[0] < max(x[0],x[2]) and tile[1] < min(x[1],x[3])]
  # if len(lines_under) > 0 and len(lines_under) % 2 == 0:
  #   return False

  # # Check if tile inside lines and lines to the left
  # lines_left = [x for x in all_lines if tile[1] > min(x[1],x[3]) and tile[1] < max(x[1],x[3]) and tile[0] > max(x[0],x[2])]
  # if len(lines_left) > 0 and len(lines_left) % 2 == 0:
  #   return False

  # Check if tile inside lines and lines to the right. HERE!!! ADDED one equal sign to make it work!?=#?!#=!€
  lines_right = [x for x in all_lines if tile[1] >= min(x[1],x[3]) and tile[1] < max(x[1],x[3]) and tile[0] < min(x[0],x[2])]
  if len(lines_right) % 2 == 0:
    return False
  else: 
    return True
  
  if len(lines_above) == 0 and len(lines_under) == 0 and len(lines_left) == 0 and len(lines_right) == 0:
    return False
  else:
    return True
  

red_tiles_set = set(red_tiles)

def get_all_corners(x1,y1,x2,y2):
  c1 = (x1,y1)
  c2 = (x1,y2)
  c3 = (x2,y1)
  c4 = (x2,y2)
  return [c1,c2,c3,c4]

part2 = 0
i = 0
tot = len(red_tiles)*len(red_tiles)/2
checked_tiles = set()
# Check corner theory
invalid_by_corner, invalid_by_side, invalid_by_area = 0,0,0

# # Add check before
for a in range(len(red_tiles)):
  for b in range(a+1,len(red_tiles)):

    red_tile_a = red_tiles[a]
    red_tile_b = red_tiles[b]

    ######## Track progress... ############
    # i+=1
    # if i % 1000 == 0: print("Iteration",i," out of ",tot,f'({round((i/tot*100),2)}%) complete)',"Invalid by corner:",invalid_by_corner,"Invalid by side:",invalid_by_side) 
    #######################################
  
    # Check valid corner
    if not all(tile_inside_polygon(x) or x in tile_on_perimeter or x in red_tiles_set for x in get_all_corners(red_tile_a[0],red_tile_a[1],red_tile_b[0],red_tile_b[1])):
      # Throw by corner
      invalid_by_corner += 1
      continue

    base = abs(red_tile_a[0]-red_tile_b[0])+1
    height = abs(red_tile_a[1]-red_tile_b[1])+1
    area = base * height

    if area > part2:
      # Check valid parameter
      perimeter = get_rectangular_perimeter(red_tile_a,red_tile_b)
      if all(tile_inside_polygon(x) or x in tile_on_perimeter or x in red_tiles_set for x in perimeter):
        # Valid rectangle. Check if largest and update
        base = abs(red_tile_a[0]-red_tile_b[0])+1
        height = abs(red_tile_a[1]-red_tile_b[1])+1
        area = base * height
        nbr_times_updated += 1
        part2 = area
      else:
        # Throw by side
        invalid_by_side += 1

    else:
      # Throw by area
      invalid_by_area += 1

print("Part 2 Answer:",part2,"after",i,"iterations where updated",nbr_times_updated,"and throw",i-nbr_times_updated) 
print("Throws by chategory:")
print("Invalid by corner:",invalid_by_corner)
print("invalid_by_area:",invalid_by_area)
print("invalid_by_side:",invalid_by_side)

# 0123456789
# .............. 0
# ..X....#XXX#.. 1
# .......X...X.. 2
# ..#XXXX#...X.. 3
# ..X....@...X.. 4 
# ..#XXXXXX#.X.. 5
# .........X.X.. 6
# .........#X#.. 7
# .............. 8


# 0123456789
# ....................0
# ..X....#XXX#........1
# .......X...X....... 2
# ..#XXXX#.@.#XX#.#X#.3
# ..X...........X.X.X.4 
# ..#XXXXXX#....#X#.X.5
# .........X........X.6
# ..X......#XXXXXXXX#. 7
# .............. 8